# Final Data Science Capstone

# Final Data Science Capstone

## Introduction - Business Problem

In this project, we will explore London, the most multicultural city in the United kingdom and its financial capital.

We explore places around London and discuss what makes London unique! London is truly a multicultural city with various restaurants like African restuarants, Chinese restuarants, German resturants and many more.

The beautiful places around London is a reason why you need to visit London. Lonson is known for its vibrant social life and it is a good place to start business. London is a city where businesses thrive. There are many amazing businesses around London.

First, we determine the places to start a restaurant in London.

and why London is a good place to live. London's vibrant social life is due to its many bars, restaurants, museums, pubs, and many more.

We use the postal district dataset given in the folowing link https://en.wikipedia.org/wiki/London_postal_district. The link provides the postal district code, postal town, coverage and local authority area(s) of postal districts of London.

We use the Foursquare API to explore the venues given the location of the London's districts.

Finally, we use K-means clustering to cluster all the coverages(districts) in London. We then group then base on the clusters.

## Data Section

In order to achieve our objectives stated in the Introduction. We use the postcode areas dataset in the folowing link https://en.wikipedia.org/wiki/London_postal_district. We scrap using pandas.

The postcode areas in london are N, NW, SW, SE, W, WC, E and EC postcode areas. For each postcode area, we collect data on all the postcode districts, post town, coverage and local authority area(s) in the area. We merge all the 8 postcode areas to generate our final dataset with all coverages(distincts) given.

We remove the data points where the local authgority area(s) is non-geographic and other None values.

We use the Geopy library to find the location data for each district in our dataset.

Finally, we use the Foursquare API to explore the venues around each district using the location data provided by Geopy.



Firstly, we want to do web scraping using pandas by extracting the postal table from https://en.wikipedia.org/wiki/London_postal_district using pandas.

## Import all the necessary Libaries

In [1]:
#import necessary libaries
!pip install geocoder
import geocoder # import geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy  # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Read the table using pandas.read_html function

There are 8 post areas in London as listed in postal_areas below

In [2]:
postal_areas = [ 'N', 'NW', 'SW', 'SE', 'W', 'WC', 'E' , 'EC']
tables = []

for item in postal_areas:
    
    if item != 'EC':
        url = 'https://en.wikipedia.org/wiki/{}_postcode_area'.format(item)
        tables_pd = pd.read_html(url)
        tables.append(tables_pd[1])
        
    elif item == 'EC':
        url = 'https://en.wikipedia.org/wiki/{}_postcode_area'.format(item)
        tables_pd = pd.read_html(url)
        tables.append(tables_pd[2])
    else:
        pass
        

The number of tables in tables can be calculated as follows:

In [3]:
len(tables)

8

Merge all the eight tables together

In [4]:
result_df = pd.concat(tables)

In [5]:
result_df.head()

,Postcode district,Post town,Coverage,Local authority area(s)
0,N1,LONDON,"Northern head district: Barnsbury (part), Cano...","Hackney, Islington, Camden"
1,N1C,LONDON,Kings Cross Central,Camden
2,N1P,LONDON,PO boxes in N1 and N1C,non-geographic
3,N2,LONDON,"East Finchley district: East Finchley, Fortis ...","Barnet, Haringey"
4,N3,LONDON,"Finchley district: Finchley, Church End, Finch...",Barnet


Find the types of the columns in the dataset

In [6]:
result_df.dtypes

Postcode district          object
Post town                  object
Coverage                   object
Local authority area(s)    object
dtype: object

The number of rows and columns in our dataset can be calculated as follows

In [7]:
print('There are {} rows and {} columns in our dataset'.format(result_df.shape[0], result_df.shape[1]))

There are 184 rows and 4 columns in our dataset


Next, collect all the data points where the Local authority area(s) = non-geographic

In [8]:
nongeo_df = result_df[result_df['Local authority area(s)'] == 'non-geographic']
nongeo_df

,Postcode district,Post town,Coverage,Local authority area(s)
2,N1P,LONDON,PO boxes in N1 and N1C,non-geographic
24,N81,LONDON,Electoral Reform Services,non-geographic
1,NW1W,LONDON,PO boxes in NW1[4],non-geographic
12,NW26,LONDON,PO boxes in NW10[4],non-geographic
1,SE1P,LONDON,NaN,non-geographic
0,W1A,LONDON,PO boxes & Admail codes in W1[7],non-geographic
3,EC1P,LONDON,NaN,non-geographic
10,EC2P,LONDON,NaN,non-geographic
17,EC3P,LONDON,NaN,non-geographic
23,EC4P,LONDON,NaN,non-geographic


Next, remove all the data points where the Local authority area(s) != non-geographic and find the resulting dataset

In [9]:
result_df = result_df[result_df['Local authority area(s)'] != 'non-geographic']

In [10]:
result_df = result_df.drop('Local authority area(s)', axis = 1).dropna()

In [11]:
result_df.shape

(173, 3)

In [12]:
result_df

,Postcode district,Post town,Coverage
0,N1,LONDON,"Northern head district: Barnsbury (part), Cano..."
1,N1C,LONDON,Kings Cross Central
3,N2,LONDON,"East Finchley district: East Finchley, Fortis ..."
4,N3,LONDON,"Finchley district: Finchley, Church End, Finch..."
5,N4,LONDON,"Finsbury Park district: Finsbury Park, Manor H..."
6,N5,LONDON,"Highbury district: Highbury, Highbury Fields"
7,N6,LONDON,"Highgate district: Highgate, Hampstead Heath (..."
8,N7,LONDON,"Holloway district: Holloway, Barnsbury (part),..."
9,N8,LONDON,"Hornsey district: Hornsey, Crouch End, Haringe..."
10,N9,LONDON,"Lower Edmonton district: Lower Edmonton, Edmon..."


In [13]:
result_df['Coverage'] = result_df['Coverage'].map(lambda name: name.split(':')[0])
result_df = result_df.reset_index(drop = True)

## Use Geopy to find the location data (longitude and latitude) of all districts

In [14]:
post_data = result_df['Postcode district']
latitudes = []
longitudes = []
for post in post_data:
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode("{}, England".format(post))
    if location != None:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        latitudes.append('NA')
        longitudes.append('NA')

In [15]:
location_data = pd.DataFrame(columns = ['Postcode district','Latitude', 'Longitude'])

## **Location data is given as follows**

In [16]:
location_data['Postcode district'] = result_df['Postcode district']
location_data['Latitude'] = pd.Series(latitudes)
location_data['Longitude'] = pd.Series(longitudes)
result_df['Latitude'] = location_data['Latitude'] 
result_df['Longitude'] = location_data['Longitude']
location_data.head()

,Postcode district,Latitude,Longitude
0,N1,52.599,-2.09829
1,N1C,51.5347,-0.124548
2,N2,52.6013,-2.09711
3,N3,52.5998,-2.09297
4,N4,52.5987,-2.09266


In [17]:
result_df = result_df.reset_index(drop=True)

The resulting dataset is given below after including the location data for each district

In [18]:
result_df

,Postcode district,Post town,Coverage,Latitude,Longitude
0,N1,LONDON,Northern head district,52.599,-2.09829
1,N1C,LONDON,Kings Cross Central,51.5347,-0.124548
2,N2,LONDON,East Finchley district,52.6013,-2.09711
3,N3,LONDON,Finchley district,52.5998,-2.09297
4,N4,LONDON,Finsbury Park district,52.5987,-2.09266
5,N5,LONDON,Highbury district,51.4808,-0.474918
6,N6,LONDON,Highgate district,51.4665,-0.468849
7,N7,LONDON,Holloway district,51.4665,-0.474702
8,N8,LONDON,Hornsey district,51.4664,-0.482383
9,N9,LONDON,Lower Edmonton district,53.7958,-1.75245


In [19]:
result_df = result_df[result_df['Latitude'] != 'NA']

In [20]:
result_df.shape

(166, 5)

In [21]:
result_df

,Postcode district,Post town,Coverage,Latitude,Longitude
0,N1,LONDON,Northern head district,52.599,-2.09829
1,N1C,LONDON,Kings Cross Central,51.5347,-0.124548
2,N2,LONDON,East Finchley district,52.6013,-2.09711
3,N3,LONDON,Finchley district,52.5998,-2.09297
4,N4,LONDON,Finsbury Park district,52.5987,-2.09266
5,N5,LONDON,Highbury district,51.4808,-0.474918
6,N6,LONDON,Highgate district,51.4665,-0.468849
7,N7,LONDON,Holloway district,51.4665,-0.474702
8,N8,LONDON,Hornsey district,51.4664,-0.482383
9,N9,LONDON,Lower Edmonton district,53.7958,-1.75245


## Map of London, England with all the districts

In [22]:
address = 'London, England'

geolocator = Nominatim(user_agent="my_app")
lond_location = geolocator.geocode(address)
lond_latitude = lond_location.latitude
lond_longitude = lond_location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(lond_latitude, lond_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [23]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[lond_latitude, lond_longitude], zoom_start=10)

# add markers to map
for lat, lng, post_town, coverage in zip(result_df['Latitude'], result_df['Longitude'], result_df['Post town'], result_df['Coverage']):
    label = '{}, {}'.format(coverage, post_town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Foursquare API 
It's time to explore the locations nearby the districts using Foursquare API.
We will start with the Foursquare details.

In [24]:
CLIENT_ID = 'BL2ZUBYOFSD40CFRZHGTYVR0WZCIC2RBXY3CP52YQJYQ4B12' # your Foursquare ID
CLIENT_SECRET = 'UCAMQEVDA0WSKEGJE10YOO1ZYDR1T4KBJAD4DXG3LCMXREPC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BL2ZUBYOFSD40CFRZHGTYVR0WZCIC2RBXY3CP52YQJYQ4B12
CLIENT_SECRET:UCAMQEVDA0WSKEGJE10YOO1ZYDR1T4KBJAD4DXG3LCMXREPC


Let us explore the first coverage(district)

In [25]:
result_df.loc[0, 'Coverage']

'Northern head district'

In [26]:
# the location data for the first district 

district_latitude = result_df.loc[0, 'Latitude'] # neighborhood latitude value
district_longitude = result_df.loc[0, 'Longitude'] # neighborhood longitude value

district_name = result_df.loc[0, 'Coverage'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, district_latitude, district_longitude))

Latitude and longitude values of Northern head district are 52.5989524, -2.0982864.


In [27]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT
)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=BL2ZUBYOFSD40CFRZHGTYVR0WZCIC2RBXY3CP52YQJYQ4B12&client_secret=UCAMQEVDA0WSKEGJE10YOO1ZYDR1T4KBJAD4DXG3LCMXREPC&v=20180605&ll=52.5989524,-2.0982864&radius=500&limit=100


In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fecfece02573325d4faf643'},
 'response': {'headerLocation': 'Wolverhampton',
  'headerFullLocation': 'Wolverhampton',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 52.603452404500004,
    'lng': -2.0908914811186214},
   'sw': {'lat': 52.5944523955, 'lng': -2.1056813188813788}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fec74393d7c9b9c78e06ae6',
       'name': 'WHSmith',
       'location': {'address': 'Newcross Hospital, Wolverhampton Road',
        'lat': 52.59942265617264,
        'lng': -2.095441938657814,
        'labeledLatLngs': [{'label': 'display',
          'lat': 52.59942265617264,
          'lng': -2.095441938657814}],
        'distance': 199,
        'postalCode': 'WV

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,WHSmith,Stationery Store,52.599423,-2.095442
1,Cineworld,Movie Theater,52.596291,-2.092581
2,Heath Town Park,Park,52.596837,-2.099860
3,Eastside Coffee Shop,Coffee Shop,52.600520,-2.094384
4,Greggs,Bakery,52.601881,-2.092942


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


## **Explore all the venues in all the districts in London using the result_df dataset**

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Coverage', 
                  'Coverage Latitude', 
                  'Coverage Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
london_venues = getNearbyVenues(names=result_df['Coverage'],
                                   latitudes=result_df['Latitude'],
                                   longitudes=result_df['Longitude']
                                  )

In [98]:
print(london_venues.shape)
london_venues['Venue Category']

(6588, 7)


0                               Stationery Store
1                                  Movie Theater
2                                           Park
3                                    Coffee Shop
4                                         Bakery
5                                            Pub
6                              Indian Restaurant
7                                          Plaza
8                                 Shopping Plaza
9                      Middle Eastern Restaurant
10                                   Supermarket
11                                Breakfast Spot
12                                         Canal
13                                   Coffee Shop
14                                     Bookstore
15                                        Market
16                                           Bar
17                                Breakfast Spot
18                        Furniture / Home Store
19                                           Gym
20                  

London has many restaurants, bars, clothing stores, pubs, shopping malls, science museums, and theaters. It is indeed a good place for social life.

## **Now, we want to find districts or places in London where there are no restaurants. One of those places is a good location to start a restaurant**

The 'london_nonrestaurants_venues' table contains that list

In [90]:
london_nonrestaurants_venues = london_venues.where(~london_venues['Venue Category'].str.contains('Restaurant'))
london_nonrestaurants_venues = london_restaurants_venues.dropna().reset_index(drop=True)
print(london_nonrestaurants_venues.shape)
for district in london_nonrestaurants_venues['Coverage'].unique():
    print(district)

(5062, 7)
Northern head district
Kings Cross Central
East Finchley district
Finchley district
Finsbury Park district
Highbury district
Highgate district
Hornsey district
Lower Edmonton district
Muswell Hill district
New Southgate district
North Finchley district
Palmers Green district
Southgate district
South Tottenham district
Stoke Newington district
Upper Edmonton district
Upper Holloway district
Whetstone district
Winchmore Hill district
Wood Green district
North Western head district
Cricklewood district
Hampstead district
Hendon district
Kentish Town district
Kilburn district
Mill Hill district
St John's Wood district
The Hyde district
Willesden district
Golders Green district
Whitehall, Buckingham Palace, the Houses of Parliament and west of St James's
Northern 'Victoria' around Buckingham Gate (a street) to include both sides of part of Victoria Street
Surrounds of St James's Park tube station centred on Broadway, Queen Anne's Gate and Old Queen Street to include both sides of 

In [36]:
london_venues.groupby('Coverage').count()

,Coverage Latitude,Coverage Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Coverage,,,,,,
"A projection around Parliament Square. Nearby Westminster School to Westminster Cathedral to CCAL, University of the Arts, London (omitting all backstreets nearest to Vauxhall Bridge Road) – derivation",55,55,55,55,55,55
Abbey Wood district,3,3,3,3,3,3
All but the west of St James's,88,88,88,88,88,88
Anerley district,5,5,5,5,5,5
Balham district,61,61,61,61,61,61
Bank of England,100,100,100,100,100,100
Barbican,67,67,67,67,67,67
Barnes district,2,2,2,2,2,2
Battersea head district,42,42,42,42,42,42


In [37]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 334 uniques categories.


In [38]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Coverage'] = london_venues['Coverage'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Coverage,Accessories Store,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop

## **Districts where you can find African Restaurants in London**

In [44]:
african_venues = london_venues[london_venues['Venue Category'] == 'African Restaurant']
african_venues

,Coverage,Coverage Latitude,Coverage Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
285,South Tottenham district,51.589918,-0.099070,Le Chamarel,51.589688,-0.105921,African Restaurant
509,Wood Green district,51.592298,-0.105241,Le Chamarel,51.589688,-0.105921,African Restaurant
1819,Stockwell district,51.476685,-0.111061,Adulis,51.479486,-0.111702,African Restaurant
3293,"Fitzrovia, Tottenham Court Road",51.524002,-0.140537,African Kitchen Gallery,51.527265,-0.136386,African Restaurant
3596,Shepherds Bush district,51.504004,-0.230712,Red Sea Restaurant,51.506485,-0.235160,African Restaurant
5378,"Finsbury, Finsbury Estate (west)",51.528399,-0.106873,Spinach & Agushi,51.525288,-0.110348,African Restaurant


List of the districts where you can find african restaurants

In [89]:
for district in african_venues.Coverage:
    print(district)

South Tottenham district
Wood Green district
Stockwell district
Fitzrovia, Tottenham Court Road
Shepherds Bush district
Finsbury, Finsbury Estate (west)


## **Map of  African Restaurants in London**

In [46]:
# create map of London using latitude and longitude values
def create_venues_map(venues):
    map_london = folium.Map(location=[lond_latitude, lond_longitude], zoom_start=10)
    for lat, lng, venue, category in zip(venues['Venue Latitude'], venues['Venue Longitude'], venues['Venue'], venues['Venue Category']):
        label = '{}, {}'.format(venue, category)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
               [lat, lng],
                radius=5,
                popup=label,
               color='green',
               fill=True,
               fill_color='#3186cc',
                fill_opacity=0.7,
                              parse_html=False).add_to(map_london)  
    return map_london
create_venues_map(african_venues)

## **Districts where you can find American Restaurants in London**

In [93]:
american_venues = london_venues[london_venues['Venue Category'] == 'American Restaurant']
for district in american_venues.Coverage:
    print(district)

Tottenham district
Barnes district
Portland Place, Regent Street
Marylebone
Marylebone
Fitzrovia, Tottenham Court Road
Great Portland Street, Fitzrovia
University College London, SOAS
Covent Garden
Somerset House, Temple (west)
Woodford and South Woodford district
Non-geographic postcode district (NatWest, delivers to E1)
Non-geographic postcode district (News International, delivers to E1W)


## **Map of American Restaurants in London**

In [ ]:
create_venues_map(american_venues)

In [49]:
london_grouped = london_onehot.groupby('Coverage').mean().reset_index()
london_grouped

,Coverage,Accessories Store,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop

## Top 5 venues in each district

In [50]:
num_top_venues = 5

for hood in london_grouped['Coverage']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Coverage'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A projection around Parliament Square. Nearby Westminster School to Westminster Cathedral to CCAL, University of the Arts, London (omitting all backstreets nearest to Vauxhall Bridge Road) – derivation----
               venue  freq
0        Coffee Shop  0.13
1               Café  0.13
2     Sandwich Place  0.05
3  Outdoor Sculpture  0.05
4              Plaza  0.05


----Abbey Wood district----
               venue  freq
0       Soccer Field  0.33
1  Fish & Chips Shop  0.33
2      Grocery Store  0.33
3  Accessories Store  0.00
4             Palace  0.00


----All but the west of St James's----
     venue  freq
0    Plaza  0.06
1    Hotel  0.06
2     Café  0.05
3  Theater  0.05
4      Pub  0.03


----Anerley district----
                  venue  freq
0  Fast Food Restaurant   0.2
1                 Hotel   0.2
2           Supermarket   0.2
3     Convenience Store   0.2
4         Grocery Store   0.2


----Balham district----
               venue  freq
0        Coffee Shop  0.15
1     

                  venue  freq
0                  Café  0.06
1         Grocery Store  0.06
2           Coffee Shop  0.06
3     Fish & Chips Shop  0.06
4  Gym / Fitness Center  0.06


----Kings Cross Central----
            venue  freq
0            Café  0.08
1            Park  0.03
2  Breakfast Spot  0.03
3   Train Station  0.03
4   Movie Theater  0.03


----Kings Cross, Finsbury (west), Clerkenwell (north)----
            venue  freq
0     Coffee Shop  0.11
1  History Museum  0.06
2       Bookstore  0.04
3      Restaurant  0.04
4          Garden  0.04


----Lee district----
                  venue  freq
0                   Gym   0.5
1     Convenience Store   0.5
2     Accessories Store   0.0
3  Pakistani Restaurant   0.0
4          Perfume Shop   0.0


----Leicester Square, St. Giles----
              venue  freq
0       Coffee Shop  0.07
1            Bakery  0.06
2           Theater  0.05
3    Ice Cream Shop  0.05
4  Sushi Restaurant  0.04


----Lewisham district----
                 

                  venue  freq
0                 Hotel  0.29
1                  Park  0.29
2  Bulgarian Restaurant  0.14
3  Fast Food Restaurant  0.14
4        Breakfast Spot  0.14


----Streatham district----
         venue  freq
0          Pub  0.14
1  Supermarket  0.09
2  Coffee Shop  0.09
3    Gastropub  0.09
4     Bus Stop  0.05


----Surrounds of St James's Park tube station centred on Broadway, Queen Anne's Gate and Old Queen Street to include both sides of part of Victoria Street – derivation----
               venue  freq
0        Coffee Shop  0.12
1              Hotel  0.09
2  Outdoor Sculpture  0.07
3                Pub  0.07
4      Historic Site  0.07


----Sydenham district----
            venue  freq
0     Supermarket  0.12
1            Café  0.12
2  Sandwich Place  0.06
3            Park  0.06
4        Pharmacy  0.06


----Temple----
         venue  freq
0          Pub  0.16
1        Hotel  0.09
2  Coffee Shop  0.09
3   Restaurant  0.05
4     Wine Bar  0.04


----The Hyde

## Ten most common venues in each district

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
def ten_most_common_venues(data = london_grouped):
    
    num_top_venues = 10
    indicators = ['st', 'nd', 'rd']
    
    # create columns according to number of top venues
    
    columns = ['Coverage']

    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
            
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))
            
    # create a new dataframe
    
    coverages_venues_sorted = pd.DataFrame(columns=columns)
    
    coverages_venues_sorted['Coverage'] = data['Coverage']
    
    for ind in np.arange(data.shape[0]):
        
        coverages_venues_sorted.iloc[ind, 1:] = return_most_common_venues(data.iloc[ind, :], num_top_venues)
        
    return coverages_venues_sorted

In [53]:
coverages_venues_sorted = ten_most_common_venues(london_grouped)
coverages_venues_sorted.head()

,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A projection around Parliament Square. Nearby ...,Coffee Shop,Café,Sandwich Place,Outdoor Sculpture,Plaza,Restaurant,Historic Site,Grocery Store,Bar,Park
1,Abbey Wood district,Fish & Chips Shop,Grocery Store,Soccer Field,Film Studio,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,All but the west of St James's,Hotel,Plaza,Theater,Café,Bookstore,Pub,Coffee Shop,Art Museum,Gelato Shop,Lounge
3,Anerley district,Hotel,Grocery Store,Convenience Store,Fast Food Restaurant,Supermarket,Yoshoku Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Balham district,Coffee Shop,Pub,Indian Restaurant,Pizza Place,Italian Restaurant,Supermarket,Bakery,Café,Bar,Sandwich Place


## Clustering using KMeans ##

In [54]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Coverage', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3],
      dtype=int32)

In [60]:

# add clustering labels
#coverages_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#coverages_venues_sorted['Cluster Labels'] = coverages_venues_sorted['Cluster Labels'].astype(int)

london_merged = result_df

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(coverages_venues_sorted.set_index('Coverage'), on='Coverage')

london_merged = london_merged.dropna()
london_merged

,Postcode district,Post town,Coverage,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,N1,LONDON,Northern head district,52.599,-2.09829,3.0,Pub,Park,Bakery,Stationery Store,Movie Theater,Coffee Shop,Yoshoku Restaurant,Fish & Chips Shop,Falafel Restaurant,Farm
1,N1C,LONDON,Kings Cross Central,51.5347,-0.124548,3.0,Café,Burger Joint,Plaza,Breakfast Spot,Bar,Train Station,Chocolate Shop,Coffee Shop,Park,Movie Theater
2,N2,LONDON,East Finchley district,52.6013,-2.09711,3.0,Bakery,Stationery Store,Coffee Shop,Yoshoku Restaurant,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,N3,LONDON,Finchley district,52.5998,-2.09297,3.0,Clothing Store,Fast Food Restaurant,Sporting Goods Shop,Coffee Shop,Mexican Restaurant,Bakery,Shopping Plaza,Sandwich Place,Pharmacy,Asian Restaurant
4,N4,LONDON,Finsbury Park district,52.5987,-2.09266,3.0,Clothing Store,Supermarket,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Asian Restaurant,Bowling Alley,Sandwich Place,Mexican Restaurant,Stationery Store
5,N5,LONDON,Highbury district,51.4808,-0.474918,3.0,Rental Car Location,Bus Stop,Hotel,Gym / Fitness Center,Airport Service,Chinese Restaurant,Coffee Shop,Indian Restaurant,Film Studio,Exhibit
6,N6,LONDON,Highgate district,51.4665,-0.468849,3.0,Airport Service,Coffee Shop,Yoshoku Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
8,N8,LONDON,Hornsey district,51.4664,-0.482383,3.0,Pharmacy,Spa,Yoshoku Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
9,N9,LONDON,Lower Edmonton district,53.7958,-1.75245,3.0,Bar,Hotel,Coffee Shop,Italian Restaurant,Plaza,Pub,Clothing Store,Theater,Pharmacy,Beer Bar
10,N10,LONDON,Muswell Hill district,51.5942,-0.14311,3.0,Café,Pizza Place,Coffee Shop,Gym / Fitness Center,Grocery Store,Bakery,Japanese Restaurant,Pub,French Restaurant,Stationery Store


In [61]:
# create map
map_clusters = folium.Map(location=[lond_latitude, lond_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Coverage'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [68]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1, 2] + list(range(5, london_merged.shape[1]))]]

,Post town,Coverage,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,LONDON,Palmers Green district,0.0,Outdoor Gym,Park,Pub,Tennis Court,Golf Course,Farm,Playground,Yoshoku Restaurant,Fabric Shop,Falafel Restaurant
23,LONDON,North Western head district,0.0,Garden,Park,Café,Coffee Shop,Pedestrian Plaza,Tennis Court,Theater,Fountain,Discount Store,Distillery
54,LONDON,Mortlake district,0.0,Sports Club,Pub,Gym,Park,Bus Stop,Farmers Market,Rugby Pitch,Film Studio,Fabric Shop,Falafel Restaurant
68,LONDON,Deptford district,0.0,Pub,Cosmetics Shop,Park,Fast Food Restaurant,Platform,Café,Vietnamese Restaurant,Bus Stop,Food & Drink Shop,Food Court
69,LONDON,Eltham district,0.0,Event Service,Park,Soccer Field,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
74,LONDON,New Cross district,0.0,Park,Bus Stop,Train Station,Clothing Store,Restaurant,Electronics Store,Supermarket,Yoshoku Restaurant,Fast Food Restaurant,Fabric Shop
78,LONDON,Woolwich district,0.0,Hotel,Train Station,Park,Supermarket,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
83,LONDON,Forest Hill district,0.0,Garden,Park,Yoshoku Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
87,LONDON,West Norwood district,0.0,Brewery,Pub,Park,Yoshoku Restaurant,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
112,LONDON,West Ealing district,0.0,Hotel,Park,Fast Food Restaurant,Supermarket,Yoshoku Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Film Studio


## Cluster 2

In [96]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1, 2] + list(range(5, london_merged.shape[1]))]]

,Post town,Coverage,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
137,LONDON,Homerton district,1.0,Park,Yoshoku Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop


## Cluster 3 ##

In [70]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1, 2] + list(range(5, london_merged.shape[1]))]]

,Post town,Coverage,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,LONDON,Tottenham district,2.0,Seafood Restaurant,American Restaurant,Yoshoku Restaurant,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
53,LONDON,Barnes district,2.0,Seafood Restaurant,American Restaurant,Yoshoku Restaurant,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
146,LONDON,Woodford and South Woodford district,2.0,Seafood Restaurant,American Restaurant,Yoshoku Restaurant,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
148,LONDON,"Non-geographic postcode district (NatWest, del...",2.0,Seafood Restaurant,American Restaurant,Yoshoku Restaurant,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
149,LONDON,Non-geographic postcode district (News Interna...,2.0,Seafood Restaurant,American Restaurant,Yoshoku Restaurant,Fish & Chips Shop,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


## Cluster 4 ##

In [71]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1, 2] + list(range(5, london_merged.shape[1]))]]

,Post town,Coverage,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,Northern head district,3.0,Pub,Park,Bakery,Stationery Store,Movie Theater,Coffee Shop,Yoshoku Restaurant,Fish & Chips Shop,Falafel Restaurant,Farm
1,LONDON,Kings Cross Central,3.0,Café,Burger Joint,Plaza,Breakfast Spot,Bar,Train Station,Chocolate Shop,Coffee Shop,Park,Movie Theater
2,LONDON,East Finchley district,3.0,Bakery,Stationery Store,Coffee Shop,Yoshoku Restaurant,Fish & Chips Shop,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,LONDON,Finchley district,3.0,Clothing Store,Fast Food Restaurant,Sporting Goods Shop,Coffee Shop,Mexican Restaurant,Bakery,Shopping Plaza,Sandwich Place,Pharmacy,Asian Restaurant
4,LONDON,Finsbury Park district,3.0,Clothing Store,Supermarket,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Asian Restaurant,Bowling Alley,Sandwich Place,Mexican Restaurant,Stationery Store
5,LONDON,Highbury district,3.0,Rental Car Location,Bus Stop,Hotel,Gym / Fitness Center,Airport Service,Chinese Restaurant,Coffee Shop,Indian Restaurant,Film Studio,Exhibit
6,LONDON,Highgate district,3.0,Airport Service,Coffee Shop,Yoshoku Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
8,LONDON,Hornsey district,3.0,Pharmacy,Spa,Yoshoku Restaurant,Fish Market,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
9,LONDON,Lower Edmonton district,3.0,Bar,Hotel,Coffee Shop,Italian Restaurant,Plaza,Pub,Clothing Store,Theater,Pharmacy,Beer Bar
10,LONDON,Muswell Hill district,3.0,Café,Pizza Place,Coffee Shop,Gym / Fitness Center,Grocery Store,Bakery,Japanese Restaurant,Pub,French Restaurant,Stationery Store


## Cluster 5

In [72]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1, 2] + list(range(5, london_merged.shape[1]))]]

,Post town,Coverage,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,LONDON,The Hyde district,4.0,Office,Bus Stop,Park,Yoshoku Restaurant,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio
131,LONDON,Bow district,4.0,Market,Bus Stop,Park,Plaza,Hockey Field,Yoshoku Restaurant,Film Studio,Fabric Shop,Falafel Restaurant,Farm
132,LONDON,Chingford district,4.0,Market,Bus Stop,Park,Plaza,Hockey Field,Yoshoku Restaurant,Film Studio,Fabric Shop,Falafel Restaurant,Farm
133,LONDON,Clapton district,4.0,Athletics & Sports,Bus Stop,Market,Park,Plaza,Hockey Field,Yoshoku Restaurant,Fabric Shop,Falafel Restaurant,Farm
135,LONDON,Forest Gate district,4.0,Market,Bus Stop,Park,Plaza,Hockey Field,Yoshoku Restaurant,Film Studio,Fabric Shop,Falafel Restaurant,Farm
136,LONDON,Hackney district,4.0,Market,Bus Stop,Park,Plaza,Hockey Field,Yoshoku Restaurant,Film Studio,Fabric Shop,Falafel Restaurant,Farm
